## Group 1: Final Project - SimpleRNN on tdcsfog
### Sihe Zheng, szheng12@depaul.edu
### YunTzu Yu, yyu54@depaul.edu
### Saruul Enkhtur, senkhtur@depaul.edu
### Lukasz Grzybek, lgrzybek@depaul.edu


#### DePaul University, College of Computing and Digital Media,
#### DSC 672 DATA SCIENCE CAPSTONE


In [ ]:
pip install -q -U keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Normalization
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
import matplotlib.pyplot as plt

2024-03-14 21:49:55.326216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 21:49:58.772287: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
full_tdcsfog = pd.read_csv('full_tdcsfog.csv')
full_tdcsfog

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Medication,Age,Sex,YearsSinceDx,NFOGQ
0,0,-9.671054,-0.097687,-2.193385,0,0,0,1,69,1,4.0,22
1,1,-9.674142,-0.094618,-2.192642,0,0,0,1,69,1,4.0,22
2,2,-9.675552,-0.092203,-2.188145,0,0,0,1,69,1,4.0,22
3,3,-9.684805,-0.092936,-2.191913,0,0,0,1,69,1,4.0,22
4,4,-9.678051,-0.093119,-2.202334,0,0,0,1,69,1,4.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
7062667,4220,-9.403467,0.089003,-3.220304,0,0,0,0,65,1,22.0,10
7062668,4221,-9.404246,0.090531,-3.216584,0,0,0,0,65,1,22.0,10
7062669,4222,-9.405770,0.084380,-3.224039,0,0,0,0,65,1,22.0,10
7062670,4223,-9.403579,0.084236,-3.236686,0,0,0,0,65,1,22.0,10


In [ ]:
array_full_tdcsfog = np.array(full_tdcsfog)
X_full_tdcsfog = array_full_tdcsfog[:,1:4]
X_full_tdcsfog

array([[-9.67105447, -0.09768655, -2.19338497],
       [-9.67414208, -0.09461769, -2.19264206],
       [-9.67555185, -0.09220266, -2.18814466],
       ...,
       [-9.40577041,  0.08438003, -3.22403873],
       [-9.40357874,  0.08423561, -3.23668611],
       [-9.40503569,  0.08202739, -3.23445806]])

In [ ]:
array_full_tdcsfog = np.array(full_tdcsfog)
y_full_tdcsfog = array_full_tdcsfog[:,4:7]
y_full_tdcsfog

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
scaler = StandardScaler()
X_full_tdcsfog = scaler.fit_transform(X_full_tdcsfog)
X_full_tdcsfog = X_full_tdcsfog.reshape((X_full_tdcsfog.shape[0], 1, X_full_tdcsfog.shape[1]))

In [ ]:
X_train_tdcsfog, X_test_tdcsfog, y_train_tdcsfog, y_test_tdcsfog = train_test_split(X_full_tdcsfog, y_full_tdcsfog, test_size=0.20, random_state=0)

In [ ]:
def SimpleRNN_model(hp):
    hp_units = hp.Int('units', min_value=96, max_value=256, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-9, 1e-12])
    #hp_dropoutrate = hp.Float("dropout_rate",min_value=0.10,max_value=0.30,step=0.10)
    #hp_layers = hp.Int("n_layers",min_value=2,max_value=6,step=2)
    hp_activation = hp.Choice("activation", ["relu", "tanh", "sigmoid","softmax"])



    model = keras.Sequential()
    model.add(keras.layers.SimpleRNN(units=hp_units, activation=hp_activation, return_sequences=True, input_shape=(X_train_tdcsfog.shape[1], X_train_tdcsfog.shape[2])))
    #for i in range(1, hp_layers):
    #model.add(keras.layers.SimpleRNN(units=hp_units, activation=hp_activation,  return_sequences=True))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(96,3, activation="relu", padding='same'))
        #if hp.Boolean("dropout"):
            #model.add(keras.layers.Dropout(hp_dropoutrate))

    model.add(keras.layers.Dense(units=3, activation="relu"))



    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mse',
                  metrics=["accuracy"])
    return model

In [ ]:
tuner1 = kt.Hyperband(SimpleRNN_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     directory = "./untitled_project_1/",
                     project_name = 'proj1')

Reloading Tuner from ./untitled_project_1/proj1/tuner0.json


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
tuner1.search(X_train_tdcsfog, y_train_tdcsfog, epochs=10, validation_split=0.2, callbacks=[stop_early])


best_hps=tuner1.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('units'),best_hps.get('learning_rate'))


96 1e-12


Trial 30 Complete [00h 59m 33s]
val_accuracy: 0.409717857837677

Best val_accuracy So Far: 0.890156090259552
Total elapsed time: 1d 13h 16m 06s
96 1e-12

In [ ]:
best_hps.get('units')

96

In [ ]:
best_hps.get('learning_rate')

1e-12

In [ ]:
best_hps.get('activation')

'relu'

In [ ]:
tuner1.results_summary(num_trials=30)

Results summary
Results in ./untitled_project_1/proj1
Showing 30 best trials
Objective(name="val_accuracy", direction="max")

Trial 0021 summary
Hyperparameters:
units: 96
learning_rate: 1e-12
activation: relu
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.890156090259552

Trial 0024 summary
Hyperparameters:
units: 96
learning_rate: 1e-12
activation: relu
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0021
Score: 0.8901053667068481

Trial 0009 summary
Hyperparameters:
units: 256
learning_rate: 1e-09
activation: sigmoid
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7464080452919006

Trial 0013 summary
Hyperparameters:
units: 128
learning_rate: 1e-12
activation: sigmoid
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0000
Score: 0.7362796664237976

Trial 0016 summary
Hyperparameters:
units: 128
learning_rate: 1e-12
activation: sigmoid
tuner/ep

In [ ]:
all_trials = tuner1.get_best_hyperparameters(num_trials=30)

In [ ]:
srnn_model = tuner1.hypermodel.build(best_hps)
history = srnn_model.fit(X_train_tdcsfog, y_train_tdcsfog, epochs=10, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

print(best_epoch,)


Epoch 1/10
141254/141254 [==============================] - 354s 2ms/step - loss: 0.2403 - accuracy: 0.8214 - val_loss: 0.2644 - val_accuracy: 0.8179
Epoch 2/10
141254/141254 [==============================] - 392s 3ms/step - loss: 0.2402 - accuracy: 0.8212 - val_loss: 0.2825 - val_accuracy: 0.8202
Epoch 3/10
141254/141254 [==============================] - 349s 2ms/step - loss: 0.2402 - accuracy: 0.8211 - val_loss: 0.2695 - val_accuracy: 0.8168
Epoch 4/10
141254/141254 [==============================] - 389s 3ms/step - loss: 0.2403 - accuracy: 0.8212 - val_loss: 0.2817 - val_accuracy: 0.8176
Epoch 5/10
141254/141254 [==============================] - 499s 4ms/step - loss: 0.2402 - accuracy: 0.8212 - val_loss: 0.2627 - val_accuracy: 0.8172
Epoch 6/10
141254/141254 [==============================] - 625s 4ms/step - loss: 0.2402 - accuracy: 0.8213 - val_loss: 0.2834 - val_accuracy: 0.8172
Epoch 7/10
141254/141254 [==============================] - 677s 5ms/step - loss: 0.2402 - accuracy:

In [ ]:
hypermodel = tuner1.hypermodel.build(best_hps)


hypermodel.fit(X_train_tdcsfog, y_train_tdcsfog, epochs=10, validation_split=0.2)


2024-03-12 16:25:15.098804: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-12 16:25:15.771034: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54241308 exceeds 10% of free system memory.
2024-03-12 16:25:15.931392: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54241308 exceeds 10% of free system memory.


Epoch 1/10


2024-03-12 16:25:16.257955: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36160872 exceeds 10% of free system memory.
2024-03-12 16:25:16.281005: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36160872 exceeds 10% of free system memory.
2024-03-12 16:25:18.219197: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36160872 exceeds 10% of free system memory.


141254/141254 [==============================] - 352s 2ms/step - loss: 0.1211 - accuracy: 0.5852 - val_loss: 0.1207 - val_accuracy: 0.5801
Epoch 2/10
141254/141254 [==============================] - 585s 4ms/step - loss: 0.1211 - accuracy: 0.5852 - val_loss: 0.1225 - val_accuracy: 0.5763
Epoch 3/10
141254/141254 [==============================] - 674s 5ms/step - loss: 0.1211 - accuracy: 0.5854 - val_loss: 0.1235 - val_accuracy: 0.5830
Epoch 4/10
141254/141254 [==============================] - 707s 5ms/step - loss: 0.1211 - accuracy: 0.5852 - val_loss: 0.1225 - val_accuracy: 0.5838
Epoch 5/10
141254/141254 [==============================] - 684s 5ms/step - loss: 0.1211 - accuracy: 0.5851 - val_loss: 0.1236 - val_accuracy: 0.5725
Epoch 6/10
141254/141254 [==============================] - 676s 5ms/step - loss: 0.1211 - accuracy: 0.5853 - val_loss: 0.1227 - val_accuracy: 0.5843
Epoch 7/10
141254/141254 [==============================] - 650s 5ms/step - loss: 0.1211 - accuracy: 0.5853 - v

In [ ]:
#test set
result = hypermodel.evaluate(X_test_tdcsfog, y_test_tdcsfog)
print("[loss, accuracy]:", result)


44142/44142 [==============================] - 145s 3ms/step - loss: 0.1214 - accuracy: 0.5788
[loss, accuracy]: [0.12143625319004059, 0.5788106918334961]


# Manual Model

In [ ]:
def SimpleRNN_model(units,lr,activation):

    model = keras.Sequential()
    model.add(keras.layers.SimpleRNN(units=units, activation=activation, return_sequences=True, input_shape=(X_train_tdcsfog.shape[1], X_train_tdcsfog.shape[2])))

    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(96,3, activation="relu", padding='same'))


    model.add(keras.layers.Dense(units=3, activation="relu"))



    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='mse',
                  metrics=["accuracy"])
    return model

In [ ]:
manual_model = SimpleRNN_model(best_hps.get('units'),best_hps.get('learning_rate'),best_hps.get('activation'))

2024-03-14 21:50:27.097676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:
manual_model.fit(X_train_tdcsfog, y_train_tdcsfog,
          epochs=4,
          validation_split=0.2)

2024-03-14 21:50:27.503453: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54241308 exceeds 10% of free system memory.
2024-03-14 21:50:27.686866: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54241308 exceeds 10% of free system memory.


Epoch 1/4


2024-03-14 21:50:27.940387: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36160872 exceeds 10% of free system memory.
2024-03-14 21:50:27.980657: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36160872 exceeds 10% of free system memory.
2024-03-14 21:50:30.398214: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36160872 exceeds 10% of free system memory.


141254/141254 [==============================] - 973s 7ms/step - loss: 0.2373 - accuracy: 0.6042 - val_loss: 0.2445 - val_accuracy: 0.6000
Epoch 2/4
141254/141254 [==============================] - 1041s 7ms/step - loss: 0.2372 - accuracy: 0.6042 - val_loss: 0.2494 - val_accuracy: 0.5990
Epoch 3/4
141254/141254 [==============================] - 718s 5ms/step - loss: 0.2372 - accuracy: 0.6041 - val_loss: 0.2489 - val_accuracy: 0.5927
Epoch 4/4
141254/141254 [==============================] - 608s 4ms/step - loss: 0.2373 - accuracy: 0.6043 - val_loss: 0.2438 - val_accuracy: 0.5997


In [ ]:
#test set
result = manual_model.evaluate(X_test_tdcsfog, y_test_tdcsfog)
print("[loss, accuracy]:", result)

44142/44142 [==============================] - 111s 3ms/step - loss: 0.2440 - accuracy: 0.5999
[loss, accuracy]: [0.24401937425136566, 0.5999308824539185]
